In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")
from langchain_groq import ChatGroq
llm = ChatGroq(model_name= "Gemma2-9b-It")

c:\Users\Dhruv Saxena\Desktop\Dhruv\GenerativeAI\LangGraph\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Dhruv Saxena\Desktop\Dhruv\GenerativeAI\LangGraph\venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
message = [{"role": "system", "content": "Uou are a helpful assistant"},
           {"role": "user", "content": "Hi how are you"}]

In [5]:
result = llm.invoke(message)
print(result.content)

As an AI, I don't have feelings, but I'm here and ready to help! How can I assist you today? 😊  




## Create a assistant

### ReACt Agent:

- Start ---> Thought ---> Action (LLM, Tool) ---> Observation ---> Final Output
- This thing work inside the loop max_turns

In [24]:
class Chatbot():
    def __init__(self, system= ""):
        self.system = system
        self.message = []
        if self.system:
            self.message.append({"role": "system", "content": system})
    def __call__(self, message):
        self.message.append({"role": "user", "content": message})
        result = self.execute()
        self.message.append({"role": "assistant",  "content": result})
        return result
    def execute(self):
        llm = ChatGroq(model_name = "Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content

In [25]:
bot = Chatbot(system= "You are a helpfull assitant")

In [26]:
bot("Hi how are you?")

"I'm doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you may have. 😊\n\nHow are you doing today?  \n\n"

In [28]:
bot.message

[{'role': 'system', 'content': 'You are a helpfull assitant'},
 {'role': 'user', 'content': 'Hi how are you?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you may have. 😊\n\nHow are you doing today?  \n\n"}]

## Defining a prompt

In [29]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

## Define the query function

In [7]:
import re

action_re = re.compile('^Action: (\w+): (.*)')

In [33]:
import httpx
def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [34]:
def simon_blog_search(q):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": q,
    })
    return response.json()[0]["text"]

In [35]:
def calculate(number):
    return eval(number)

In [37]:
calculate("2+2")

4

In [12]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [41]:
def query(question, max_turns=7):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result


In [42]:
print(query("Has Simon been to Madagascar?"))

Thought: I need to know if Simon has been to Madagascar.  I can search his blog.

Action: simon_blog_search: Madagascar
PAUSE 

 -- running simon_blog_search Madagascar
Observation: None
Thought: Simon hasn't written about Madagascar on his blog. 

Action: wikipedia: Madagascar
PAUSE 

 -- running wikipedia Madagascar
Observation: None
Thought: I'm not sure how to answer that.  I don't have personal experience or access to information beyond my training data. 

Answer: I don't know if Simon has been to Madagascar. I can't access real-time information or personal details. 

Thought: I'm not sure how to answer that.  I don't have personal experience or access to information beyond my training data. 

Answer: I don't know if Simon has been to Madagascar. I can't access real-time information or personal details. 



In [43]:
print(query("Fifteen * twenty five"))

Thought: I need to calculate Fifteen * twenty five  

Action: calculate: 15 * 25 
PAUSE 

 -- running calculate 15 * 25 
Observation: 375
Answer: 375
Answer: 375


In [44]:
print(query("where is New Delhi?"))

Thought: I need to find the capital of India. 

Action: wikipedia: New Delhi 
PAUSE 

 -- running wikipedia New Delhi 
Observation: None
Thought:  Maybe I should try a different search term. 

Action: wikipedia: India
PAUSE 

 -- running wikipedia India
Observation: None
Thought: Hmm, it seems like I'm having trouble getting information from Wikipedia.  Let me try Simon's blog.

Action: simon_blog_search: India
PAUSE 

 -- running simon_blog_search India
Observation: None
Thought: It looks like I'm unable to access information from either Wikipedia or Simon's blog. I'll have to try a different approach. 

Action: calculate: 1 + 1 
PAUSE 





 -- running calculate 1 + 1 
Observation: 2
Thought: Even though I can do basic calculations, I still can't find the capital of India.  I need more reliable sources of information.

Action: wikipedia: India
PAUSE 

 -- running wikipedia India
Observation: None
Thought: I'm still not able to access information from Wikipedia.  It seems like I'm enc

In [45]:
print(query("what nasa does?"))

Thought: I wonder what NASA does.  

Action: wikipedia: NASA 
PAUSE 

 -- running wikipedia NASA 
Observation: None
Thought:  That's strange, wikipedia: NASA didn't give me any results. Let me try searching Simon's blog.

Action: simon_blog_search: NASA 
PAUSE 

 -- running simon_blog_search NASA 
Observation: None
Thought: Hmm, it seems like I can't find any information about NASA from either Wikipedia or Simon's blog. I'll have to assume that NASA is not a topic covered in those sources. 

Answer: I'm unable to find information about NASA from my available sources.  Would you like me to try a different question? 

Thought: Hmm, it seems like I can't find any information about NASA from either Wikipedia or Simon's blog. I'll have to assume that NASA is not a topic covered in those sources. 

Answer: I'm unable to find information about NASA from my available sources.  Would you like me to try a different question? 



In [46]:
bot("Hello, how are you?")

"I'm doing well, thank you!  \n\nIt seems like you're asking me how I am a few times.  Is there anything else I can help you with? 😊  \n"